In [1]:
import pandas as pd

df = pd.read_excel(r"C:\Users\brend\OneDrive - Stonehill College\All Cape League Trackman.xlsx")

In [6]:
import pandas as pd

def assign_zone_bucket(row):
    side = row['PlateLocSide']
    height = row['PlateLocHeight']
    
    if height < 1.5: v_zone = "Down"
    elif height <= 3.5: v_zone = "Mid"
    else: v_zone = "Up"

    if side < -0.25: h_zone = "In"
    elif side > 0.25: h_zone = "Away"
    else: h_zone = "Middle"
    
    return f"{v_zone}-{h_zone}"

df["ZoneBucket"] = df.apply(assign_zone_bucket, axis=1)

misses = df[(df["PitchCall"] == "StrikeSwinging") & (df["Batter"] == "Schmidt, Kent")].copy()

sequences = []

for idx, miss in misses.iterrows():
    pitcher = miss["Pitcher"]
    pitcher_side = miss.get("PitcherThrows", "Unknown")
    pitch_type = miss["TaggedPitchType"]
    if pitch_type == "Fastball":
        pitch_type = "Four-Seam"
    zone_bucket = miss["ZoneBucket"]
    date = miss["Date"]

    later_pitches = df[(df["Date"] == date) & 
                       (df["Pitcher"] == pitcher) &
                       (df["TaggedPitchType"] == miss["TaggedPitchType"]) &
                       (df["ZoneBucket"] == zone_bucket)]
    
    later_pitches = later_pitches[later_pitches.index > idx]
    
    if not later_pitches.empty:
        next_pitch = later_pitches.iloc[0]
        next_type = next_pitch["TaggedPitchType"]
        if next_type == "Fastball":
            next_type = "Four-Seam"
        
        sequences.append({
            "Pitcher": pitcher,
            "PitcherThrows": pitcher_side,
            "MissDate": miss["Date"],
            "MissPitchType": pitch_type,
            "MissZone": miss["ZoneBucket"],
            "MissCall": miss["PitchCall"],
            "MissResult": miss["PlayResult"],
            "MissKorBB": miss["KorBB"] if miss["KorBB"] != "Undefined" else "",
            "MissExitSpeed": miss.get("ExitSpeed", None),
            "MissExitAngle": miss.get("Angle", None),
            "NextDate": next_pitch["Date"],
            "NextPitchType": next_type,
            "NextZone": next_pitch["ZoneBucket"],
            "NextCall": next_pitch["PitchCall"],
            "NextResult": next_pitch["PlayResult"],
            "NextKorBB": next_pitch["KorBB"] if next_pitch["KorBB"] != "Undefined" else "",
            "NextExitSpeed": next_pitch.get("ExitSpeed", None),
            "NextExitAngle": next_pitch.get("Angle", None),
        })

seq_df = pd.DataFrame(sequences)

pd.set_option("display.max_rows", None)

for i, seq in enumerate(sequences, 1):
    miss_date_str = pd.to_datetime(seq['MissDate']).strftime('%Y-%m-%d')
    next_date_str = pd.to_datetime(seq['NextDate']).strftime('%Y-%m-%d')
    
    print(f"\n=== Sequence {i} ===")
    print(f"Pitcher: {seq['Pitcher']} | PitcherSide: {seq['PitcherThrows']}")
    
    print("\n-- Whiff Pitch --")
    print(f"Date: {miss_date_str}")
    print(f"PitchType: {seq['MissPitchType']} | Zone: {seq['MissZone']}")
    print(f"PitchCall: {seq['MissCall']} | PlayResult: {seq['MissResult']}")
    if seq['MissKorBB']:
        print(f"Kor/BB: {seq['MissKorBB']}")
    if seq['MissCall'] == "InPlay":
        print(f"ExitSpeed: {seq['MissExitSpeed']} | LaunchAngle: {seq['MissExitAngle']}")
    
    print("\n-- Next Pitch (Same game, pitcher, pitch, and location) --")
    print(f"Date: {next_date_str}")
    print(f"PitchType: {seq['NextPitchType']} | Zone: {seq['NextZone']}")
    print(f"PitchCall: {seq['NextCall']} | PlayResult: {seq['NextResult']}")
    if seq['NextKorBB']:
        print(f"Kor/BB: {seq['NextKorBB']}")
    if seq['NextCall'] == "InPlay":
        print(f"ExitSpeed: {seq['NextExitSpeed']} | LaunchAngle: {seq['NextExitAngle']}")


=== Sequence 1 ===
Pitcher: Bowie, Rhys | PitcherSide: Left

-- Whiff Pitch --
Date: 2025-08-01
PitchType: Four-Seam | Zone: Mid-In
PitchCall: StrikeSwinging | PlayResult: Undefined

-- Next Pitch (Same game, pitcher, pitch, and location) --
Date: 2025-08-01
PitchType: Four-Seam | Zone: Mid-In
PitchCall: BallCalled | PlayResult: Undefined

=== Sequence 2 ===
Pitcher: Chmielewski, Tom | PitcherSide: Left

-- Whiff Pitch --
Date: 2025-07-08
PitchType: Four-Seam | Zone: Up-In
PitchCall: StrikeSwinging | PlayResult: Undefined

-- Next Pitch (Same game, pitcher, pitch, and location) --
Date: 2025-07-08
PitchType: Four-Seam | Zone: Up-In
PitchCall: BallCalled | PlayResult: Undefined

=== Sequence 3 ===
Pitcher: Colucci, Joe | PitcherSide: Right

-- Whiff Pitch --
Date: 2025-07-15
PitchType: Cutter | Zone: Mid-In
PitchCall: StrikeSwinging | PlayResult: Undefined

-- Next Pitch (Same game, pitcher, pitch, and location) --
Date: 2025-07-15
PitchType: Cutter | Zone: Mid-In
PitchCall: InPlay | P